<a href="https://colab.research.google.com/github/josepmatias/ONE/blob/main/comissionar_alarmes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
pip install pyxlsb

In [3]:
#actualizar dados ficheiros, folhas e colunas

ficheiro_alarmes="ONE_Alarmes_mar24_versão inicial.xlsb"
folha_alarmes="mar24"
col_daf="Analise DAF"
col_horario="carga horária"

In [29]:
#criar tabelas de comissionamento alarmes

comissionar_full=pd.DataFrame({"carga horária": [8]*8, "alarmes": range(1, 9), "upfront": [0, 0.25, 0.5, 1, 1, 1.1, 1.1, 1.2], \
                "atingimento_gco":[0.5, 0.8, 0.9, 1, 1, 1.3, 1.3, 1.5]})


comissionar_half=pd.DataFrame({"carga horária": [4]*10, "alarmes": range(1, 11), "upfront":[0.25, 1, 1.1, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2], \
                "atingimento_gco":[0.8, 1, 1.3, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5]})

print(comissionar_half.dtypes)
comissionar_half

#format the numbers in a percentage manner
#comissionar_half.loc[:, "upfront"].apply(lambda i: i*100).astype("int").astype("str").apply(lambda j: j + "%")


carga horária        int64
alarmes              int64
upfront            float64
atingimento_gco    float64
dtype: object


,carga horária,alarmes,upfront,atingimento_gco
0,4,1,0.25,0.8
1,4,2,1.00,1.0
2,4,3,1.10,1.3
3,4,4,1.20,1.5
4,4,5,1.20,1.5
5,4,6,1.20,1.5
6,4,7,1.20,1.5
7,4,8,1.20,1.5
8,4,9,1.20,1.5
9,4,10,1.20,1.5


In [6]:
#carregar ficheiro dos alarmes

from google.colab import files
uploaded = files.upload()

alarmes_db=pd.read_excel(ficheiro_alarmes, sheet_name=folha_alarmes)

Saving ONE_Alarmes_mar24_versão inicial.xlsb to ONE_Alarmes_mar24_versão inicial.xlsb


In [ ]:
#adicionar coluna "gco", "ib upfront" (valor de alarme com ou sem sdd), "ib gco" (ib upfront*(% upfront-1)), "valor final"


alarmes_db.head()

,Mês de Fecho,Order Item,Chave,SFID,Comissionamento,Sub-Área de Negócio,Service Account,Customer Account (SA),CA&UA,Alojamento Id,...,MSISDN SA Mobile (a),Campanha wise,"Campanha outbound (por CAV, Campanha Final)",carga horária,Notas DS,Final,Analise DAF,Pagar,Conclusão SDD,Análise SDD
0,45352,1-KO8FSLI,20544240583580073949,B.C01.UP1.581,ONE,ALARME,S965521480,C232270001,C23227000110017527331,792389,...,?,up_alarmes_excl,up_alarmes_excl,4,NaN,ok,Comissionado Alarme,Sim,Comissionar alarme com valor com sdd,OK
1,45352,1-KNG0D4T,20544240583570052743,B.C01.UP1.351,ONE,ALARME,S965486403,C260815401,C26081540110017484864,4839165,...,?,up_alarmes_excl,up_alarmes_excl,8,NaN,ok,Comissionado Alarme,Sim,Comissionar alarme com valor com sdd,OK
2,45352,1-KP39CKC,40168940583780161754,L.G01.UP1.625,ONE,ALARME,S965564908,C324886201,C32488620110017567477,43676,...,?,up_alarmes_excl,up_alarmes_excl,4,NaN,ok,Comissionado Alarme,Sim,Comissionar alarme com valor sem sdd,NOK - Flag Débito Direto = Não
3,45352,1-KPO0LHQ,20544240583570052743,B.C01.UP1.351,ONE,ALARME,S965595084,C336819301,C33681930110017618136,1096290,...,?,up_alarmes_excl,up_alarmes_excl,8,NaN,ok,Comissionado Alarme,Sim,Comissionar alarme com valor sem sdd,NOK - Flag Débito Direto = Não
4,45352,1-KO1UTJP,40168940583780161754,L.G01.UP1.625,ONE,ALARME,S965513582,C666590901,C66659090110017519474,3205525,...,?,up_alarmes_excl,up_alarmes_excl,4,NaN,ok,Comissionado Alarme,Sim,Comissionar alarme com valor com sdd,OK


In [7]:
#criar coluna cav na dataframe de alarmes

def extrair_cav (x):

  dicionario_cav={"entidade": 6, "delegacao": (6, 12), "cav": 12}
  for i, j in dicionario_cav.items():
    if i=="entidade":
      entidade=x[:j]
    elif i=="delegacao":
      start_index, end_index=j
      delegacao=x[start_index:end_index]
    else:
      cav=x[j:]
  return cav

alarmes_db["cav"]=alarmes_db.loc[:, "Chave"].apply(extrair_cav)



In [8]:
alarmes_db2=alarmes_db.loc[alarmes_db[col_daf]=="Comissionado Alarme", :]

alarmes_grupos=pd.DataFrame(alarmes_db2.value_counts(["cav", col_horario])).reset_index()

alarmes_grupos.columns=["cav", "carga horária", "alarmes"]

alarmes_grupos


,cav,carga horária,alarmes
0,80161754,4,6
1,80048024,8,4
2,70052743,8,2
3,80073949,4,2


In [52]:
for i in comissionar_half.loc[:, ["upfront", "atingimento_gco"]].iterrows():
  val, val2 =float(i)
  print(float(val2))



TypeError: float() argument must be a string or a real number, not 'tuple'

In [37]:
#alarmes_grupos.loc[alarmes_grupos["carga horária"]==4, :].merge(comissionar_half, on="carga horária", how="left")


#alarmes_grupos.loc[alarmes_grupos["carga horária"]==4, :].join(comissionar_half, on="carga horária", how="left", rsuffix="alarmes")

def quant_alarmes (df, df2, horario):
  upfront=[]
  atingimento=[]

  df_registo=df.loc[df["carga horária"]==horario, ["alarmes"]]
  for i, j in df_registo.items():
    upfront.append(float(df2.loc[df2["alarmes"]==j, "upfront"]))
    atingimento.append(float(df2.loc[df2["alarmes"]==j, "upfront"]))

  return upfront, atingimento

quant_alarmes(alarmes_grupos, comissionar_half, 4)




ValueError: Can only compare identically-labeled Series objects